In [1]:
import os
from PIL import Image
from augmentations import flip_image, rotate_and_zoom_image, adjust_brightness, add_noise

# Create output directory for augmented images
output_dir = "./Data/augmented_image_single_test"
os.makedirs(output_dir, exist_ok=True)

# Load a test image
image_path = "./Data/fits_filtered2/tic6.png"  
if not os.path.exists(image_path):
    raise FileNotFoundError(f"Test image not found at {image_path}")

img = Image.open(image_path)

# Apply augmentations and save each as a separate file
augmentations = [
    ("horizontal_flip", flip_image(img, mode='horizontal')),
    ("vertical_flip", flip_image(img, mode='vertical')),
    ("diagonal_flip", flip_image(img, mode='diagonal')),
    ("rotate_zoom", rotate_and_zoom_image(img, angle=8, zoom_factor=1.5)),
    ("brightened", adjust_brightness(img, factor=1.5)),
    ("noisy", add_noise(img, noise_level=0.15))
]

for name, aug_img in augmentations:
    output_path = os.path.join(output_dir, f"{name}.png")
    aug_img.save(output_path)

print(f"Augmented images saved to {output_dir}")


Augmented images saved to ./Data/augmented_image_single_test
